# Thinking
### Thinking1：常见的规划问题都包括哪些？
    线性规划（LP）、整数线性规划（ILP）、混合整数规划（MIP）、路径问题规划（VRP）
### Thinking2：常用的规划工具包都有哪些？
    现使用scripy、pulp、ortools
### Thinking3：RFM模型的原理是怎样的
    RFM为Recency(距离最近的一次交易)、Frequency(交易频率)、Monetary(交易金额)的缩写，再既定规则下计算R、F、M综合分值越高，客户价值越高。
    通常分为八类：重要价值客户(R:高，F:高，M:高)、重要发展用户(R:高，F:低，M:高)、重要保持用户(R:低，F:高，M:高)、重要挽留用户(R:低，F:低，M:高)、一般价值用户(R:高，F:高，M:低)、一般发展用户(R:高，F:低，M:低)、一般保持用户(R:低，F:高，M:低)、一般挽留用户(R:低，F:低，M:低)

# Action2

In [1]:
import pulp
import numpy as np

In [2]:
# 定义数据，共6个亩地、4个品种农作物，怎么种使得总收益最大。
costs = np.array([[500, 550, 630, 1000, 800, 700],
                      [800, 700, 600, 950, 900, 930],
                      [1000, 960, 840, 650, 600, 700],
                      [1200, 1040, 980, 860, 880, 780]])
# 计划播种面积
max_plant = [76, 88, 96, 40]
# 拥有地块面积
max_cultivation = [42, 56, 44, 39, 60, 59]

row = len(costs)
col = len(costs[0])
print(f'row:{row} col:{col}')

row:4 col:6


In [3]:
# 定义初始问题
prob = pulp.LpProblem('Transportation', sense=pulp.LpMaximize)
# 定义相关变量
var = [[pulp.LpVariable(f'x{i}{j}', lowBound=0, cat=pulp.LpInteger) for j in range(col)] for i in range(row)]
var

[[x00, x01, x02, x03, x04, x05],
 [x10, x11, x12, x13, x14, x15],
 [x20, x21, x22, x23, x24, x25],
 [x30, x31, x32, x33, x34, x35]]

In [4]:
# 递归展开列表
flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]
print('var=', var)
print('flatten=', flatten)
print('var[1]', var[1])

var= [[x00, x01, x02, x03, x04, x05], [x10, x11, x12, x13, x14, x15], [x20, x21, x22, x23, x24, x25], [x30, x31, x32, x33, x34, x35]]
flatten= <function <lambda> at 0x7f23b85d8320>
var[1] [x10, x11, x12, x13, x14, x15]


In [5]:
# 定义目标函数 （单价* {ij}运量）
prob += pulp.lpDot(flatten(var), costs.flatten())

# 定义约束条件
for i in range(row):
    prob += pulp.lpSum(var[i]) <= max_plant[i]
for j in range(col):
    prob += pulp.lpSum([var[i][j] for i in range(row)]) <= max_cultivation[j]

In [6]:
# 使用线性规划计算结果
prob.solve()
result = {'objective':pulp.value(prob.objective), \
          'var': [[pulp.value(var[i][j]) for j in range(col)] for i in range(row)]}

In [8]:
# 输出结果
print(f'最大值为{result["objective"]}')
print('各变量的取值为：')
print(result['var'])

最大值为284230.0
各变量的取值为：
[[0.0, 0.0, 6.0, 39.0, 31.0, 0.0], [0.0, 0.0, 0.0, 0.0, 29.0, 59.0], [2.0, 56.0, 38.0, 0.0, 0.0, 0.0], [40.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
